# Calculator Agent with Strands SDK

This notebook demonstrates:
- Creating a simple calculator agent
- Tool invocation in real-time
- ReAct loop (Reasoning + Acting) pattern

In [3]:
# Install required packages
!pip install strands-agents strands-agents-tools boto3 -q

In [25]:
from strands import Agent
from strands import tool
import json

## Define Calculator Tools

In [26]:
@tool
def add(a: float, b: float) -> float:
    """Add two numbers
    
    Args:
        a: first number to add
        b: second numeber to add
    """
    return a + b

@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers

    Args:
        a: first number to multiple
        b: second numeber to multiply
    """
    return a * b

@tool
def divide(a: float, b: float) -> float:
    """Divide two numbers

    Args:
        a: first number to divide
        b: second numeber to divide
    """
    if b == 0:
        return "Error: Division by zero"
    return a / b

## Create Agent with Tools

In [27]:
agent = Agent(
    name="calculator",
    model="us.amazon.nova-2-lite-v1:0",
    tools=[add, multiply, divide],
    system_prompt="You are a calculator assistant. Use the provided tools to perform calculations."
)

## Run Agent - Observe ReAct Loop

The agent will:
1. **Reason**: Understand the calculation needed
2. **Act**: Invoke appropriate tools
3. **Observe**: Process tool results
4. **Respond**: Provide final answer

In [28]:
# Simple calculation
response = agent("What is 15 + 27?")
print("Response:", response)


Tool #1: add
15 + 27 = 42Response: 15 + 27 = 42



In [29]:
# Multi-step calculation
response = agent("Calculate (10 + 5) * 3")
print("Response:", response)


Tool #2: add

Tool #3: multiply
(10 + 5) * 3 = 45Response: (10 + 5) * 3 = 45



In [30]:
# Complex calculation
response = agent("What is 100 divided by 4, then multiply the result by 7?")
print("Response:", response)


Tool #4: divide

Tool #5: multiply
100 divided by 4 is 25, and 25 multiplied by 7 is 175. 

So the final result is **175**.Response: 100 divided by 4 is 25, and 25 multiplied by 7 is 175. 

So the final result is **175**.



## Whats the ReAct loop

In agents, this is abstracted and may include other optimizations.

In [67]:
import boto3
from pprint import pprint
bedrock = boto3.client('bedrock-runtime', region_name='us-east-1')

system = [
    {"text": 
         """You are a helpful assistant. To help you complete the task you have the following tools:

         {
            tool_name: add,
            parameters_requires:
                {
                    a = firt number to add,
                    b = second number to add
                },
            returns: addition of the two numbers
        },
        {
            tool_name: multiply,
            parameters_requires:
                {
                    a = firt number to multiply,
                    b = second number to multiply
                },
            returns: mulitplication of the two numbers
        },
        {
            tool_name: divide,
            parameters_requires:
                {
                    a: firt number to divide,
                    b: second number to divide
                },
            returns: division of the two numbers
        }

        When requiring a tool to execute you need to return the response in valid json in the following format and only one tool:

        {
            "tool_name": <name of the tool to execute>,
            "parameters":
            {
                "a": <first value>,
                "b": <second value>
            }
        }

        Once you have the final answer, return the answer in the following format:
        
        {
            final_answer: <answer>
        }
        
""" }]

prompt = "What is 100 divided by 4, then multiply the result by 7?"
# Prepare the message
messages = [
    {
        "role": "user",
        "content": [{"text": prompt}]
    }
]

# Call Converse API with Nova Lite
response = bedrock.converse(
    modelId="us.amazon.nova-lite-v1:0",
    messages=messages,
    system=system,
    inferenceConfig={
        "maxTokens": 512,
        "temperature": 0.1,
        "topP": 0.1
    }
)

output = response['output']['message']['content'][0]['text']

messages.append({
    "role": "assistant",
    "content": [{"text": output}]
})

try:
    while True:
        tool_request = json.loads(output)

        if "tool_name" not in tool_request:
            break

        tool_name = tool_request['tool_name']

        a = tool_request['parameters']['a']
        b = tool_request['parameters']['b']
        
        if tool_name == "add":
            answer = add(a, b)
        elif tool_name == "multiply":
            answer = multiply(a, b)
        elif tool_name == "divide":
            answer = divide(a, b)

        messages.append({
            "role": "user",
            "content": [{"text": f"The result of tool {tool_name} is {answer}"}]
        })

        response = bedrock.converse(
            modelId="us.amazon.nova-lite-v1:0",
            messages=messages,
            system=system,
            inferenceConfig={
                "maxTokens": 512,
                "temperature": 0.1,
                "topP": 0.1
            }
        )
        output = response['output']['message']['content'][0]['text']

        messages.append({
            "role": "assistant",
            "content": [{"text": output}]
        })
except Exception as e:
    print("ERROR", e)
    pass
pprint(messages)
print("="*90)
print(json.loads(output)['final_answer'])

[{'content': [{'text': 'What is 100 divided by 4, then multiply the result by '
                       '7?'}],
  'role': 'user'},
 {'content': [{'text': '{\n'
                       '    "tool_name": "divide",\n'
                       '    "parameters": {\n'
                       '        "a": 100,\n'
                       '        "b": 4\n'
                       '    }\n'
                       '}'}],
  'role': 'assistant'},
 {'content': [{'text': 'The result of tool divide is 25.0'}], 'role': 'user'},
 {'content': [{'text': '{\n'
                       '    "tool_name": "multiply",\n'
                       '    "parameters": {\n'
                       '        "a": 25.0,\n'
                       '        "b": 7\n'
                       '    }\n'
                       '}'}],
  'role': 'assistant'},
 {'content': [{'text': 'The result of tool multiply is 175.0'}],
  'role': 'user'},
 {'content': [{'text': '{\n    "final_answer": 175.0\n}'}],
  'role': 'assistant'}]
175.0
